<a href="https://colab.research.google.com/github/shachi-i/Volatility-forecasting/blob/main/lstm_garch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas_datareader import data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers


import datetime



from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score





In [57]:




from keras.models import Model
from keras.layers import *



import tensorflow as tf

In [58]:
import yfinance as yf

tickers = ["^N225", "^GDAXI"]  # Nikkei 225 (Japan) + DAX 30 (Germany)
data = yf.download(tickers, start="2010-01-01", end="2025-01-01")['Close']


/tmp/ipython-input-643132691.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2010-01-01", end="2025-01-01")['Close']
[*********************100%***********************]  2 of 2 completed


In [59]:
import numpy as np
returns = np.log(data / data.shift(1)).dropna()
realized_vol = returns.rolling(window=21).std()  # 1-month rolling vol


bench mark statistical model

In [60]:
!pip install arch

Statistical forecasting for N225 (JP)

In [61]:
from arch import arch_model

garch = arch_model(returns['^N225'], vol='Garch', p=1, q=1)
res = garch.fit()
preds = res.forecast(horizon=1, reindex=False)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 9.249693234114344e+20
Iteration:      2,   Func. Count:     19,   Neg. LLF: 2.772335523364103e+19
Optimization terminated successfully    (Exit mode 0)
            Current function value: -10098.748206934717
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [62]:
egarch11 = arch_model(returns['^N225'], vol='EGarch', p=1, q=1, dist='normal')
res_egarch = egarch11.fit(update_freq=5, disp='off')


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


generate volatility forecasts

In [63]:
forecast_garch = res.forecast(horizon=1, reindex=False)
forecast_egarch = res_egarch.forecast(horizon=1, reindex=False)


vol_garch = np.sqrt(forecast_garch.variance.values[-1, :])[0]
vol_egarch = np.sqrt(forecast_egarch.variance.values[-1, :])[0]



In [64]:
realized = returns[-len(returns):] ** 2
pred_garch = vol_garch ** 2

In [65]:
rmse_g = np.sqrt(np.mean((pred_garch - realized) ** 2))


In [66]:
mae_g = np.mean(np.abs(pred_garch - realized))


In [67]:
qlike_g = np.mean(np.log(pred_garch ) + (realized / pred_garch ))


In [68]:
e_realized = returns[-len(returns):] ** 2
e_predicted = vol_egarch ** 2
rmse_egarch = np.sqrt(np.mean((e_predicted - e_realized) ** 2))
mae_egarch = np.mean(np.abs(e_predicted - e_realized))
qlike_egarch = np.mean(np.log(e_predicted) + (e_realized / e_predicted))

In [69]:
import pandas as pd

comparison = pd.DataFrame(
    [[rmse, mae, qlike]],
    index=["GARCH(1,1)","EGARCH(1,1)"],
    columns=["RMSE", "MAE", "QLIKE"]
)
print(comparison)



                 RMSE       MAE  QLIKE
GARCH(1,1)   0.003216  0.002414    NaN
EGARCH(1,1)  0.003216  0.002414    NaN


Statistical forecasting for DAX 30 (DE)

In [70]:
from arch import arch_model

garch1 = arch_model(returns['^GDAXI'], vol='Garch', p=1, q=1)
res1 = garch.fit()
preds1 = res.forecast(horizon=1, reindex=False)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 9.249693234114344e+20
Iteration:      2,   Func. Count:     19,   Neg. LLF: 2.772335523364103e+19
Optimization terminated successfully    (Exit mode 0)
            Current function value: -10098.748206934717
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [71]:
egarch11_1 = arch_model(returns['^GDAXI'], vol='EGarch', p=1, q=1, dist='normal')
res_egarch_1 = egarch11_1.fit(update_freq=5, disp='off')

/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001521. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [72]:
forecast_garch_1 = res1.forecast(horizon=1, reindex=False)
forecast_egarch_1 = res_egarch_1.forecast(horizon=1, reindex=False)


vol_garch1 = np.sqrt(forecast_garch_1.variance.values[-1, :])[0]
vol_egarch1 = np.sqrt(forecast_egarch_1.variance.values[-1, :])[0]


In [73]:
realized1 = returns[-len(returns):] ** 2
predicted1 = vol_garch1 ** 2
rmse1 = np.sqrt(np.mean((predicted1 - realized1) ** 2))
mae1 = np.mean(np.abs(predicted1 - realized1))
qlike1 = np.mean(np.log(predicted1) + (realized1 / predicted1))
e_realized1 = returns[-len(returns):] ** 2
e_predicted1 = vol_egarch1 ** 2
rmse_egarch1 = np.sqrt(np.mean((e_predicted1 - e_realized1) ** 2))
mae_egarch1 = np.mean(np.abs(e_predicted1 - e_realized1))
qlike_egarch1 = np.mean(np.log(e_predicted1) + (e_realized1 / e_predicted1))

In [74]:
import pandas as pd

comparison = pd.DataFrame(
    [[rmse, mae, qlike]],
    index=["GARCH(1,1)","EGARCH(1,1)"],
    columns=["RMSE", "MAE", "QLIKE"]
)
print(comparison)


                 RMSE       MAE  QLIKE
GARCH(1,1)   0.003216  0.002414    NaN
EGARCH(1,1)  0.003216  0.002414    NaN


splitting data into train and test sets

Linear Regression

LSTM

In [75]:
import numpy as np

def make_sequences(df, feature_cols, seq_len=21):
    X, y = [], []
    for i in range(seq_len, len(df)-1):
        window = df.iloc[i-seq_len:i][feature_cols].values
        X.append(window)
        y.append(df.iloc[i+1]['rv'])   # next-day realized variance
    return np.array(X), np.array(y)


In [76]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Create 10 years of synthetic data (approx 2500 days)
n = 2500
dates = pd.date_range(start="2010-01-01", periods=n, freq="B")

# Synthetic price path using geometric brownian motion
mu = 0.0002
sigma = 0.01
returns = np.random.normal(mu, sigma, n)
price = 100 * (1 + returns).cumprod()

df_price = pd.DataFrame({
    'Date': dates,
    '^N225': price
}).set_index('Date')


In [77]:
feat = df_price.copy()

feat['^N225_r'] = feat['^N225'].pct_change()
feat['^N225_r_lag_1'] = feat['^N225_r'].shift(1)
feat['^N225_r_lag_2'] = feat['^N225_r'].shift(2)

feat['rv'] = feat['^N225_r']**2
feat['rv_21'] = feat['rv'].rolling(21).sum()

# Fake GARCH variance (for now use rolling variance)
feat['garch_var'] = feat['^N225_r'].rolling(21).var()

feat = feat.dropna()


In [78]:
feature_cols = [
    '^N225_r',
    '^N225_r_lag_1',
    '^N225_r_lag_2',
    'rv_21',
    'garch_var'
]


In [79]:
X, y = make_sequences(feat, feature_cols, seq_len=21)

split = int(0.8 * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [80]:

from sklearn.preprocessing import StandardScaler

n_features = X_train.shape[2]

scaler = StandardScaler().fit(X_train.reshape(-1, n_features))

X_train_s = scaler.transform(X_train.reshape(-1, n_features)).reshape(X_train.shape)
X_test_s  = scaler.transform(X_test.reshape(-1, n_features)).reshape(X_test.shape)


In [81]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(21, n_features)),
    layers.GRU(64, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='relu') # Added relu activation to ensure non-negative predictions
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_3 (GRU)                     │ (None, 64)             │        13,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,745 (61.50 KB)

 Trainable params: 15,745 (61.50 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
history = model.fit(
    X_train_s, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0032 - val_loss: 2.0495e-07
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 9.5442e-07 - val_loss: 2.9787e-08
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 2.6308e-07 - val_loss: 2.9787e-08
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 1.3475e-07 - val_loss: 2.9787e-08
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 4.1379e-08 - val_loss: 2.9787e-08
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 3.3566e-08 - val_loss: 2.9787e-08
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 2.9488e-08 - val_loss: 2.9787e-08
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 2.8596e-08 - val_loss: 2.9787e-08
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 3.0814e-08 - val_loss: 2.9787e-08
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.0030e-08 - val_loss: 2.9787e-08
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 3.0098e-08 - 

In [83]:
pred_lstm = model.predict(X_test_s).flatten()


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [84]:
def compute_metrics(pred, real):
    rmse = np.sqrt(np.mean((pred - real)**2))
    mae  = np.mean(np.abs(pred - real))
    qlike = np.mean(np.log(pred) + real / pred)
    return rmse, mae, qlike

rmse, mae, qlike = compute_metrics(pred_lstm, y_test)

print("RMSE:", rmse)
print("MAE:", mae)
print("QLIKE:", qlike)


RMSE: 0.00032076644870542454
MAE: 0.00010490007434956306
QLIKE: nan


/tmp/ipython-input-3436056194.py:4: RuntimeWarning: divide by zero encountered in log
  qlike = np.mean(np.log(pred) + real / pred)
/tmp/ipython-input-3436056194.py:4: RuntimeWarning: divide by zero encountered in divide
  qlike = np.mean(np.log(pred) + real / pred)
/tmp/ipython-input-3436056194.py:4: RuntimeWarning: invalid value encountered in add
  qlike = np.mean(np.log(pred) + real / pred)


Hybrid garch lstm

In [85]:
from arch import arch_model

returns = feat['^N225_r']

garch = arch_model(returns, vol='Garch', p=1, q=1).fit(disp='off')
garch_f = garch.forecast(horizon=1, reindex=False)

#feat['garch_var'] = garch_f.variance.iloc[:,0]
garch_var = []
for t in range(len(returns)):
    train = returns.iloc[:t+1]
    m = arch_model(train, vol='Garch', p=1, q=1).fit(disp='off')
    f = m.forecast(horizon=1).variance.iloc[-1,0]
    garch_var.append(f)

feat['garch_var'] = garch_var

feat['garch_resid'] = feat['rv'] - feat['garch_var']


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.769e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.771e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/loca

In [86]:
feat['r'] = feat['^N225_r']
feat['r_lag_1'] = feat['r'].shift(1)
feat['r_lag_2'] = feat['r'].shift(2)

feat['rv'] = feat['r'] ** 2
feat['rv_21'] = feat['r'].rolling(21).std() ** 2


In [87]:
feat = feat.dropna()


In [88]:
feature_cols = ['r','r_lag_1','r_lag_2','rv_21','garch_var']
X, y = make_sequences(feat, feature_cols, seq_len=21)

In [89]:
#feature_cols = ['^N225_r','^N225_r_lag_1','^N225_r_lag_2','rv_21','garch_var']


#X, y = make_sequences(feat, feature_cols, seq_len=21)


In [90]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [91]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train.reshape(-1, X_train.shape[2]))

X_train_s = scaler.transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test_s  = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)


In [92]:
real = y_test.copy()


In [93]:
from tensorflow.keras import layers, models

model1 = models.Sequential([
    layers.Input(shape=(21, len(feature_cols))),
    layers.GRU(64),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model1.compile(optimizer='adam', loss='mse')
model1.fit(X_train_s, y_train, epochs=20, batch_size=32, verbose=1)


Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0039
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.7062e-04
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.1183e-04
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.0221e-04
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.9960e-04
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.9891e-05
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.5232e-04
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.8771e-04
Epoch 9/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.1980e-04
Epoch 10/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 2.6448e-05
Epoch 11/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 2.7595e-05
Epoch 12/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 3.6251e-05
Epoch 13/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.8999e-05
Epoch 14/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 4.6120e-05
Epoch 15/20
61/61 ━

In [94]:
pred_hybrid = model1.predict(X_test_s).flatten()

import numpy as np

rmse1 = np.sqrt(np.mean((pred_hybrid - y_test) ** 2))
print("Hybrid LSTM-GARCH RMSE:", rmse1)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
Hybrid LSTM-GARCH RMSE: 0.0035512498596496442


In [95]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def eval_metrics(true, pred):
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred)
    return mse, rmse, mae, r2

models = {
    "GARCH": pred_garch,
    "LSTM": pred_lstm,
    "Hybrid": pred_hybrid
}

for name, pred in models.items():
    mse, rmse, mae, r2 = eval_metrics(real, pred)
    print(f"\n{name} MODEL")
    print(f"MSE  : {mse:.6f}")
    print(f"RMSE : {rmse:.6f}")
    print(f"MAE  : {mae:.6f}")
    print(f"R²   : {r2:.6f}")


InvalidParameterError: The 'y_pred' parameter of mean_squared_error must be an array-like. Got np.float64(8.414341857069554e-05) instead.